In [52]:
import os
from glob import glob
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import pickle

import warnings
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)

In [6]:
# Path to dataset
PATH = '/cdtshared/wearables/students/group5/'

# Features from biobank
features = pd.read_pickle(PATH+"imputed_dataset.pkl")

In [42]:
features_of_interest = list(set(list(features.columns)) - set(['acc.overall.avg']))

Y = features['acc.overall.avg']
X = features[features_of_interest]

In [43]:
# identify the categorical features
categorical_features = []
for columns in list(X.columns):
    if features[columns].dtype=='object':
        categorical_features.append(columns)

In [44]:
# Encode the categorical variables
X_enc = pd.get_dummies(X, columns=categorical_features)
Y = Y.to_numpy()

In [45]:
# Split into training and testing, 70:20:10
X_train, X_test, y_train, y_test = train_test_split(X_enc, Y, test_size=0.3)

In [46]:
# Split into training and validation
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.1/(0.1 + 0.2))

In [47]:
print("Shape of training set:", X_train.shape)
print("Shape of validation set:", X_val.shape)
print("Shape of test set:", X_test.shape)

Shape of training set: (69589, 1139)
Shape of validation set: (19882, 1139)
Shape of test set: (9942, 1139)


In [50]:
pca = PCA(n_components=15)
pca.fit(X_train)
X_pca_train = pca.transform(X_train)
X_pca_val = pca.transform(X_val)
X_pca_test = pca.transform(X_test)

In [53]:
# with open('/cdtshared/wearables/students/group5/XtrainPCA.pkl','wb') as f:
#     pickle.dump(X_pca_train, f)
# with open('/cdtshared/wearables/students/group5/XvalPCA.pkl','wb') as f:
#     pickle.dump(X_pca_val, f)
# with open('/cdtshared/wearables/students/group5/XtestPCA.pkl','wb') as f:
#     pickle.dump(X_pca_test, f)